In [ ]:
import pandas as pd 
import os
import cv2
import fnmatch
%matplotlib inline
import matplotlib.image as mpimg
import keras
from keras import backend as K
from keras.models import Sequential,load_model,Model
from keras.losses import binary_crossentropy
from keras.optimizers import Adam,RMSprop,Nadam
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input,SeparableConv2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.applications import VGG19,VGG16,ResNet50,Xception
import numpy as np
from keras.utils import np_utils
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
path='../input/train-scene classification/train-scene classification/train/'
plt.figure(figsize=(50, 50))
for i in range(0, 6):
    plt.subplot(2, 3, i+1)
    image = cv2.imread(path+str(i)+".jpg")
    plt.imshow(image)
plt.tight_layout()
plt.show()

In [ ]:
path='../input/train-scene classification/train-scene classification/'
df = pd.read_csv(path+'train.csv')
tot_x = df['image_name']
tot_y = df['label']
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    l = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out
x = chunkIt(tot_x,4)
y = chunkIt(tot_y,4)

In [ ]:
def extract_x(l):
    path='../input/train-scene classification/train-scene classification/'
    x = []
    for f in l:
        img = cv2.imread(path+'train/'+f)
        img = cv2.resize(img,(150,150))
        x.append(img)
    lb = (int)(0.6*len(x))
    return np.array(x[:lb]),np.array(x[lb:])

In [ ]:
def extract_y(l):
    path='../input/train-scene classification/train-scene classification/'
    y = []
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(l)
    encoded_Y = encoder.transform(l)
    # convert integers to dummy variables (i.e. one hot encoded)
    y = np_utils.to_categorical(encoded_Y)
    lb = (int)(0.6*len(y))
    return np.array(y[:lb]),np.array(y[lb:])

In [ ]:
def model():
    image_input = Input(shape=(150,150, 3))
    model=ResNet50(weights='imagenet',include_top=True, input_tensor=image_input)
    last_layer = model.get_layer('fc1000').output
    out = Dense(1000, activation='relu', name='fc10003')(last_layer)
    out = Dense(500, activation='relu', name='fc10002')(last_layer)
    out = Dense(6, activation='softmax',name ='fc1001')(out)
    custom_model = Model(input=image_input,output=out)
    for layer in custom_model.layers[:-10]:
        layer.trainable = False
    
    return custom_model

In [ ]:
model = model()
model.summary()

In [ ]:
#et=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
rt=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [ ]:
for i in range(0,4):
    train_x,valid_x = extract_x(x[i])
    train_y,valid_y = extract_y(y[i])
    train_x,train_y = shuffle(train_x,train_y,random_state=0)
    valid_x,valid_y = shuffle(valid_x, valid_y,random_state=0)
    datagen = ImageDataGenerator(rescale=1 / 255.0,rotation_range=20,zoom_range=0.05,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.05,horizontal_flip=True,vertical_flip=True,fill_mode="nearest")
    valAug = ImageDataGenerator(rescale=1 / 255.0)
    valAug.fit(valid_x)
    datagen.fit(train_x)
    history=model.fit_generator(datagen.flow(train_x, train_y, batch_size=32),validation_data=valAug.flow(valid_x,valid_y,batch_size=32),steps_per_epoch=len(train_x) // 32, epochs=20,validation_steps=len(valid_x) // 32,verbose=1,callbacks=[rt])
    print("Batch "+str(i+1)+" completed")

In [ ]:
scores = model.evaluate(valid_x, valid_y, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1]*100)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()